<a href="https://colab.research.google.com/github/SuraRzayeva/PopularityOfOnlineContent/blob/main/Thesis_(Main).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## B.Sc. Thesis - Sura Rzayeva

# Predicting the future popularity of online content based on early observations.

#### Importing libraries

In [ ]:
import pandas as pd

Loading dataset from google drive

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

raw_df = pd.read_csv('gdrive/My Drive/Thesis/dataset.csv') 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Analyzing the dataset

In [ ]:
raw_df.isna().sum()

title                 3
genres           111727
cats              34375
meta_cname            0
program_start         0
program_end           0
total_users           0
dtype: int64

In [ ]:
df = raw_df.dropna(axis=0)

In [ ]:
df.isna().sum()

title            0
genres           0
cats             0
meta_cname       0
program_start    0
program_end      0
total_users      0
dtype: int64

## Feature generation

In [ ]:
df.program_start = pd.to_datetime(df.program_start)
df.program_end = pd.to_datetime(df.program_end)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [ ]:
# Duration, year, month, days, hour
df['duration'] = df.program_end - df.program_start
df['duration'] = df['duration'].dt.total_seconds().div(60).astype(int)

df['year'] = df.program_start.dt.year
df['month'] = df.program_start.dt.month
df['days'] = df.program_start.dt.day
df['hour'] = df.program_start.dt.hour

# Weekday
week_day = []
for i in df.program_start:
    week_day_ = i.strftime("%w")
    week_day.append(week_day_)

df['week_day'] = week_day
df['week_day'] = df['week_day'].apply(lambda x: int(x) + 1)

# Is_weekend
weekend = []
for i in df.week_day:
    if int(i) > 5:
        weekend.append(True)
    else:
        weekend.append(False)

df['is_weekend'] = weekend

# Season
season = []
for i in df.month:
    if int(i) == 1 or int(i) == 12 or int(i) == 2:
      season.append('Winter')
    elif int(i) == 3 or int(i) == 4 or int(i) == 5:
      season.append('Spring')
    elif int(i) == 6 or int(i) == 7 or int(i) == 8:
      season.append('Summer')
    elif int(i) == 9 or int(i) == 10 or int(i) == 11:
      season.append('Autumn')

df['season'] = season

# Prime time
prime_time = []

for h in df.hour:
    if int(h) > 20:
        prime_time.append(True)
    else:
        prime_time.append(False)

df['prime_time'] = prime_time

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

In [ ]:
# Adding holiday days and names
from datetime import date
import calendar
import holidays

de_holidays = holidays.DE()

df['is_holiday'] = df['program_start'].apply(lambda x: x in de_holidays)
df['holiday_name'] = df['program_start'].apply(lambda x: de_holidays.get(x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


## Cleaning the dataset

In [ ]:
df = df.drop(columns=["program_start", "program_end", "holiday_name"])

In [ ]:
len(df.genres.unique())

2487

In [ ]:
# Simplifying the 'genres' feature
len(list(df.genres.unique()))

genres_list = []

for row in df['genres']:
    genres_list.append(row.split(':')[0])

genres_list
df['genres'] = genres_list

len(list(df.genres.unique()))

Sport = ["Soccer", "Boxing", "Winter Sports", "Olympics", "Football", "Athletics", "Equestrian", "Aquatic Sports", "Handball",  "Darts", "Curling", "US Sports", "Rugby", "Tennis", "Golf", "Team Sports", "Hurling", "Basketball", "Martial Arts", "Cycling", "Ice Hockey", "Gaelic Games", "Volleyball", "Extreme Sports", "Other Sports", "eSports", "Cricket"]
Music = ["Music", "Music Show", "Pop & Rock", "Folk Music", "Classical Music", "Music Clips", "Music Movie", "Musical", "Jazz"]
Children = ["Children's Show", "Children's Movie", "Children's Series", "Children's News"]
Hobby = ["Home Improvement", "Fishing", "Home & Garden"]
Reality = ["Reality", "Courtroom Reality Show"]
Talk = ["Talk Show", "Interview"]
Youth = ["Youth", "Coming-of-Age"]
News = ["News", "Report", "Disasters"]
Show = ["Entertainment show", "Award show", "Game Show"]
Fantasy = ["Fantasy", "Fairy Tale"]
Cars = ["Motor Sports", "Motor & Traffic"]
Health = ["Health", "Hospital"]

modified_genres = []

for value in df.genres:
    if value in Sport:
      modified_genres.append('Sport')
    elif value in Music:
      modified_genres.append('Music')
    elif value in Children:
      modified_genres.append('Children')
    elif value in Hobby:
      modified_genres.append('Hobby')
    elif value in Reality:
      modified_genres.append('Reality')
    elif value in Talk:
      modified_genres.append('Talk')
    elif value in Youth:
      modified_genres.append('Youth')
    elif value in News:
      modified_genres.append('News')
    elif value in Show:
      modified_genres.append('Show')
    elif value in Fantasy:
      modified_genres.append('Fantasy')
    elif value in Cars:
      modified_genres.append('Cars')
    elif value in Health:
      modified_genres.append('Health')
    else:
      modified_genres.append(value)
  
df['genres'] = modified_genres

In [ ]:
len(df.genres.unique())

53

## Categorizing the target variable

In [ ]:
popularity_cat = []

for row in df['total_users']:
    if row >= 20000:
        popularity_cat.append('star')
    elif row < 20000 and row >= 7000:
        popularity_cat.append('popular')
    elif row < 7000 and row >= 2000:
        popularity_cat.append('medium')
    else:
        popularity_cat.append('unpopular')

df['popularity_cat'] = popularity_cat

df['popularity_cat'].unique()

array(['star', 'popular', 'medium', 'unpopular'], dtype=object)

In [ ]:
df['popularity_cat'].value_counts()

unpopular    450459
medium       372409
popular       79903
star          20393
Name: popularity_cat, dtype: int64

In [ ]:
df.to_csv('cleaned_data.csv', index = False)

## Encoding nominal values

In [ ]:
df.head(5)

,title,genres,cats,meta_cname,total_users,duration,year,month,days,hour,week_day,is_weekend,season,prime_time,is_holiday,popularity_cat
2,Schlag den Star,Show,Entertainment,pro7_deutschland,231073,245,2022,1,29,19,7,True,Winter,False,False,star
3,Bundestagswahl 2021,News,Information,ard,216943,165,2021,9,26,15,1,False,Autumn,False,False,star
4,Schlag den Star,Show,Entertainment,pro7_deutschland,214322,240,2021,11,27,19,7,True,Autumn,False,False,star
5,Schlag den Star,Show,Entertainment,pro7_deutschland,211244,245,2021,12,18,19,7,True,Winter,False,False,star
6,"Wetten, dass..?",Show,Entertainment,zdf,205743,180,2021,11,6,19,7,True,Autumn,False,False,star


In [ ]:
from sklearn.preprocessing import LabelEncoder

ord_enc = LabelEncoder()

cat_column_names = ['title', 'genres', 'cats', 'meta_cname', 'year', 'month', 'days', 'hour', 'week_day', 'is_weekend', 'season', 'prime_time', 
                    'is_holiday', 'popularity_cat']

for cat_col in cat_column_names:
    cat_col_name_encode = str(cat_col) + str('_encoded')
    df[cat_col_name_encode] = ord_enc.fit_transform(df[[cat_col]])

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
df

,title,genres,cats,meta_cname,total_users,duration,year,month,days,hour,...,year_encoded,month_encoded,days_encoded,hour_encoded,week_day_encoded,is_weekend_encoded,season_encoded,prime_time_encoded,is_holiday_encoded,popularity_cat_encoded
2,Schlag den Star,Show,Entertainment,pro7_deutschland,231073,245,2022,1,29,19,...,1,0,28,19,6,1,3,0,0,2
3,Bundestagswahl 2021,News,Information,ard,216943,165,2021,9,26,15,...,0,8,25,15,0,0,0,0,0,2
4,Schlag den Star,Show,Entertainment,pro7_deutschland,214322,240,2021,11,27,19,...,0,10,26,19,6,1,0,0,0,2
5,Schlag den Star,Show,Entertainment,pro7_deutschland,211244,245,2021,12,18,19,...,0,11,17,19,6,1,3,0,0,2
6,"Wetten, dass..?",Show,Entertainment,zdf,205743,180,2021,11,6,19,...,0,10,5,19,6,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1050033,Le journal de la nuit,Weather,Information,canalF_la_chaine_meteo,1001,510,2021,8,9,20,...,0,7,8,20,1,0,2,0,0,3
1050034,Tagesschau,Magazine,Information,das_erste_de,1001,10,2022,5,24,13,...,1,4,23,13,2,0,1,0,0,3
1050035,Infomercial,Home Shopping,Information,tele-5,1001,15,2022,5,19,23,...,1,4,18,23,4,0,1,1,0,3
1050036,Mountain Men - Überleben in der Wildnis,Nature,Series,kabel1_doku,1001,40,2021,10,14,12,...,0,9,13,12,4,0,0,0,0,3


In [ ]:
df.sample(5)

,title,genres,cats,meta_cname,total_users,duration,year,month,days,hour,...,year_encoded,month_encoded,days_encoded,hour_encoded,week_day_encoded,is_weekend_encoded,season_encoded,prime_time_encoded,is_holiday_encoded,popularity_cat_encoded
783323,Zu Gast im Süden Afrikas: Von Botswana bis Kap...,Nature,Documentaries,br-alpha,1381,45,2021,8,8,9,...,0,7,7,9,0,0,2,0,0,3
124501,Ibiza - Ein Urlaub mit Folgen!,Humour,Movies,einsfestival,6489,80,2021,9,18,21,...,0,8,17,21,6,1,0,1,0,0
57600,Fußball,Sport,Sports,dsf,11115,60,2021,7,24,17,...,0,6,23,17,6,1,2,0,0,1
67967,Emmerdale,Reality,Series,eir_virgin_media_one,9962,30,2022,6,14,18,...,1,5,13,18,2,0,2,0,0,1
368222,Big Pacific,Nature,Documentaries,zdf-info,2797,45,2021,10,17,9,...,0,9,16,9,0,0,0,0,0,0


In [ ]:
# Slicing the dataset and keeping only encoded features

df_ml = df.copy()

encoded_columns = df_ml.columns.str.contains('encoded') 
duration = df_ml.columns.str.contains('duration')

df_ml = df_ml.loc[:, encoded_columns + duration]

df_ml.columns

Index(['duration', 'title_encoded', 'genres_encoded', 'cats_encoded',
       'meta_cname_encoded', 'year_encoded', 'month_encoded', 'days_encoded',
       'hour_encoded', 'week_day_encoded', 'is_weekend_encoded',
       'season_encoded', 'prime_time_encoded', 'is_holiday_encoded',
       'popularity_cat_encoded'],
      dtype='object')

In [ ]:
df_ml.to_csv('encoded_data.csv', index = False)

## Splitting data into test-train sets (keeping all the features)

In [ ]:
df_ml = pd.read_csv('encoded_data.csv')

In [ ]:
df_ml.head(2)

,duration,title_encoded,genres_encoded,cats_encoded,meta_cname_encoded,year_encoded,month_encoded,days_encoded,hour_encoded,week_day_encoded,is_weekend_encoded,season_encoded,prime_time_encoded,is_holiday_encoded,popularity_cat_encoded
0,245,42178,41,1,294,1,0,28,19,6,1,3,0,0,2
1,165,6297,33,2,24,0,8,25,15,0,0,0,0,0,2


In [ ]:
from sklearn.model_selection import train_test_split

X = df_ml.iloc[:, 0:14]
y = df_ml.iloc[:, 14]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Implementation

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, auc

##### Decision Tree

###### Feature importance showed us that the below mentioned features are not important for decision tree: "season_encoded", "year_encoded", "is_weekend_encoded", "prime_time_encoded", "is_holiday_encoded". However, removing them didn't improve the results, on the opposite resulted in a lower score. Therefore the main test split is used for this model: X_train, X_test, y_train, y_test

In [ ]:
from sklearn import tree

model_DT = tree.DecisionTreeClassifier(criterion="entropy")
model_DT.fit(X_train, y_train)

DecisionTreeClassifier(criterion='entropy')

In [ ]:
DT_prediction = model_DT.predict(X_test)

In [ ]:
accuracy_score(y_test, DT_prediction)

0.8106459841956747

In [ ]:
precision_score(y_test, DT_prediction, average='weighted')

0.8111187154382495

In [ ]:
recall_score(y_test, DT_prediction, average='weighted')

0.8106459841956747

In [ ]:
f1_score(y_test, DT_prediction, average='weighted')

0.8108225346768012

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, DT_prediction, pos_label=3)
auc(fpr, tpr)

0.8445918541801853

##### Random Forest

###### In RF, removing non-important features increase the results, therefore they will be removed and a new splits will be used. X_train_RF, X_test_RF, y_train_RF, t_test_RF.

In [ ]:
import pandas as pd

RF_df_ml = pd.read_csv('encoded_data.csv')

RF_df_ml 

,duration,title_encoded,genres_encoded,cats_encoded,meta_cname_encoded,year_encoded,month_encoded,days_encoded,hour_encoded,week_day_encoded,is_weekend_encoded,season_encoded,prime_time_encoded,is_holiday_encoded,popularity_cat_encoded
0,245,42178,41,1,294,1,0,28,19,6,1,3,0,0,2
1,165,6297,33,2,24,0,8,25,15,0,0,0,0,0,2
2,240,42178,41,1,294,0,10,26,19,6,1,0,0,0,2
3,245,42178,41,1,294,0,11,17,19,6,1,3,0,0,2
4,180,52843,41,1,457,0,10,5,19,6,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923159,510,29430,50,2,58,0,7,8,20,1,0,2,0,0,3
923160,10,45731,28,2,95,1,4,23,13,2,0,1,0,0,3
923161,15,23939,24,2,394,1,4,18,23,4,0,1,1,0,3
923162,40,35030,32,5,217,0,9,13,12,4,0,0,0,0,3


In [ ]:
RF_df_ml = RF_df_ml.drop(columns=["season_encoded", "year_encoded", "is_weekend_encoded", 'prime_time_encoded', 'is_holiday_encoded'])

RF_df_ml

,duration,title_encoded,genres_encoded,cats_encoded,meta_cname_encoded,month_encoded,days_encoded,hour_encoded,week_day_encoded,popularity_cat_encoded
0,245,42178,41,1,294,0,28,19,6,2
1,165,6297,33,2,24,8,25,15,0,2
2,240,42178,41,1,294,10,26,19,6,2
3,245,42178,41,1,294,11,17,19,6,2
4,180,52843,41,1,457,10,5,19,6,2
...,...,...,...,...,...,...,...,...,...,...
923159,510,29430,50,2,58,7,8,20,1,3
923160,10,45731,28,2,95,4,23,13,2,3
923161,15,23939,24,2,394,4,18,23,4,3
923162,40,35030,32,5,217,9,13,12,4,3


In [ ]:
from sklearn.model_selection import train_test_split

X = RF_df_ml.iloc[:, 0:9]
y = RF_df_ml.iloc[:, 9]

X_train_RF, X_test_RF, y_train_RF, y_test_RF = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_RF = RandomForestClassifier(n_estimators=130, criterion="entropy")
model_RF.fit(X_train_RF, y_train_RF)

RandomForestClassifier(criterion='entropy', n_estimators=130)

In [ ]:
RF_prediction = model_RF.predict(X_test_RF)

In [ ]:
accuracy_score(y_test_RF, RF_prediction)

0.8373476030828725

In [ ]:
precision_score(y_test_RF, RF_prediction, average='weighted')

0.8365498878761919

In [ ]:
recall_score(y_test_RF, RF_prediction, average='weighted')

0.8373476030828725

In [ ]:
f1_score(y_test_RF, RF_prediction, average='weighted')

0.836834820734636

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test_RF, RF_prediction, pos_label=3)

auc(fpr, tpr)

0.8704268965759361

##### Gradient Boosting

###### In Boosting models, removing less features reduced the accuracy score, therefore i use the data splits, where all the features are existing: X_train, X_test, y_train, y_test

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model_GB = GradientBoostingClassifier(loss="deviance", n_estimators=300)
model_GB.fit(X_train, y_train)

GB_prediction = model_GB.predict(X_test)

In [ ]:
accuracy_score(y_test, GB_prediction)

0.7124078577502396

In [ ]:
precision_score(y_test, GB_prediction, average='weighted')

0.7072113873990569

In [ ]:
recall_score(y_test, GB_prediction, average='weighted')

0.7124078577502396

In [ ]:
f1_score(y_test, GB_prediction, average='weighted')

0.703038470655147

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, GB_prediction, pos_label=3)
auc(fpr, tpr)

0.779131933530858

##### XGBoost

In [ ]:
from xgboost import XGBClassifier

model_XGB = XGBClassifier()
model_XGB.fit(X_train, y_train, verbose=False)

XGB_prediction = model_XGB.predict(X_test)

In [ ]:
accuracy_score(y_test, XGB_prediction)

0.6196833718782666

In [ ]:
precision_score(y_test, XGB_prediction, average='weighted')

0.6057697199170529

In [ ]:
recall_score(y_test, XGB_prediction, average='weighted')

0.6196833718782666

In [ ]:
f1_score(y_test, XGB_prediction, average='weighted')

0.5907725421725523

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, XGB_prediction, pos_label=3)
auc(fpr, tpr)

0.7063133972630516

##### CatBoost

In [ ]:
# Please change the runtime type on the menu above to GPU 

In [ ]:
!pip install catboost

from catboost import CatBoostClassifier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.6 MB 82 kB/s 


In [ ]:

model_CB = CatBoostClassifier()
model_CB.fit(X_train, y_train, verbose = False)

CB_prediction = model_CB.predict(X_test)

In [ ]:
accuracy_score(y_test, CB_prediction)

0.7915648881835858

In [ ]:
precision_score(y_test, CB_prediction, average='weighted')

0.7894589003804278

In [ ]:
recall_score(y_test, CB_prediction, average='weighted')

0.7915648881835858

In [ ]:
f1_score(y_test, CB_prediction, average='weighted')

0.7895667430210036

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, CB_prediction, pos_label=3)
auc(fpr, tpr)

0.8369704637696604

##### Neural Networks

###### In Neural Networks model dropping the below mentioned features helped increasing the accuracy score. Therefore, here the data splits will be used, which don't include these features. X_train_NN, X_test_NN, y_train_NN, y_test_NN

In [ ]:
NN_df_ml = df_ml.copy()

NN_df_ml 

,duration,title_encoded,genres_encoded,cats_encoded,meta_cname_encoded,year_encoded,month_encoded,days_encoded,hour_encoded,week_day_encoded,is_weekend_encoded,season_encoded,prime_time_encoded,is_holiday_encoded,popularity_cat_encoded
0,245,42178,41,1,294,1,0,28,19,6,1,3,0,0,2
1,165,6297,33,2,24,0,8,25,15,0,0,0,0,0,2
2,240,42178,41,1,294,0,10,26,19,6,1,0,0,0,2
3,245,42178,41,1,294,0,11,17,19,6,1,3,0,0,2
4,180,52843,41,1,457,0,10,5,19,6,1,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923159,510,29430,50,2,58,0,7,8,20,1,0,2,0,0,3
923160,10,45731,28,2,95,1,4,23,13,2,0,1,0,0,3
923161,15,23939,24,2,394,1,4,18,23,4,0,1,1,0,3
923162,40,35030,32,5,217,0,9,13,12,4,0,0,0,0,3


In [ ]:
NN_df_ml  = NN_df_ml .drop(columns=["month_encoded", "year_encoded", "is_weekend_encoded", 'prime_time_encoded', 'is_holiday_encoded', 'days_encoded', 'season_encoded', 'title_encoded', 'week_day_encoded'])
NN_df_ml 

,duration,genres_encoded,cats_encoded,meta_cname_encoded,hour_encoded,popularity_cat_encoded
0,245,41,1,294,19,2
1,165,33,2,24,15,2
2,240,41,1,294,19,2
3,245,41,1,294,19,2
4,180,41,1,457,19,2
...,...,...,...,...,...,...
923159,510,50,2,58,20,3
923160,10,28,2,95,13,3
923161,15,24,2,394,23,3
923162,40,32,5,217,12,3


In [ ]:
from sklearn.model_selection import train_test_split

X = NN_df_ml.iloc[:, 0:5]
y = NN_df_ml.iloc[:, 5]

X_train_NN, X_test_NN, y_train_NN, y_test_NN = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from tensorflow import keras
import tensorflow as tf

In [ ]:
X_train_NN.shape

(738531, 5)

In [ ]:
X_test_NN.shape

(184633, 5)

In [ ]:
model_NN = keras.Sequential([
                keras.layers.Flatten(),
                keras.layers.Dense(128,activation = tf.nn.relu), 
                keras.layers.Dense(64,activation = tf.nn.sigmoid), 
                keras.layers.Dense(32,activation = tf.nn.sigmoid),                          
                keras.layers.Dense(10,activation = tf.nn.softmax)
                ])

model_NN.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics =['accuracy'])

model_NN.fit(X_train_NN, y_train_NN, epochs=17)

Epoch 1/17
23080/23080 [==============================] - 79s 3ms/step - loss: 0.9783 - accuracy: 0.5061
Epoch 2/17
23080/23080 [==============================] - 66s 3ms/step - loss: 0.9238 - accuracy: 0.5430
Epoch 3/17
23080/23080 [==============================] - 59s 3ms/step - loss: 0.9006 - accuracy: 0.5551
Epoch 4/17
23080/23080 [==============================] - 62s 3ms/step - loss: 0.8875 - accuracy: 0.5636
Epoch 5/17
23080/23080 [==============================] - 58s 3ms/step - loss: 0.8780 - accuracy: 0.5702
Epoch 6/17
23080/23080 [==============================] - 62s 3ms/step - loss: 0.8702 - accuracy: 0.5757
Epoch 7/17
23080/23080 [==============================] - 64s 3ms/step - loss: 0.8641 - accuracy: 0.5792
Epoch 8/17
23080/23080 [==============================] - 60s 3ms/step - loss: 0.8590 - accuracy: 0.5831
Epoch 9/17
23080/23080 [==============================] - 61s 3ms/step - loss: 0.8552 - accuracy: 0.5846
Epoch 10/17
23080/23080 [==============================

In [ ]:
import numpy as np

NN_prediction = model_NN.predict(X_test_NN, batch_size=64, verbose=1)

NN_prediction_bool = np.argmax(NN_prediction, axis=1)

2885/2885 [==============================] - 5s 2ms/step


In [ ]:
accuracy_score(y_test_NN, NN_prediction_bool)

0.5918497776670476

In [ ]:
precision_score(y_test_NN, NN_prediction_bool, average='weighted')

0.5801420330484964

In [ ]:
recall_score(y_test_NN, NN_prediction_bool, average='weighted')

0.5918497776670476

In [ ]:
f1_score(y_test_NN, NN_prediction_bool, average='weighted')

0.5778724587828488

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test_NN, NN_prediction_bool, pos_label=2)
auc(fpr, tpr)

0.4396526135739317